**İmports and Reading Dataset**

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind


pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

dataframe_control = pd.read_excel("ab_testing.xlsx" , sheet_name="Control Group")
dataframe_test = pd.read_excel("ab_testing.xlsx" , sheet_name="Test Group")

df_control = dataframe_control.copy()
df_test = dataframe_test.copy()

**General İnformation of Dataset**

In [2]:
def information(df):
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

information(df_control)
information(df_test)

###############################    Shape  ##################################
(40, 4)
###############################    Types  ##################################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
###############################    Head   ##################################
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
###############################    Tail   ##################################
     Impression      Click  Purchase    Earning
35 132064.21900 3747.15754 551.07241 2256.97559
36  86409.94180 4608.25621 345.04603 1781.35769
37 123678.93423 3649.07379 476.16813 2187.72122
38 101997.49410 4736.35337 474.61354 2254.56383
39 121085.88122 4285.17861 590.40602 1289.30895
############

**Concatenation of Datasets**

In [3]:
df_control["group"] = "Control"
df_test["group"] = "Test"

df = pd.concat([df_control,df_test], axis=0,ignore_index=False)
df.head()

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,Control
1,98050.45193,3382.86179,315.08489,1742.80686,Control
2,82696.02355,4167.96575,458.08374,1797.82745,Control
3,109914.40040,4910.88224,487.09077,1696.22918,Control
4,108457.76263,5987.65581,441.03405,1543.72018,Control


**Defining the A/B Test Hypothesis**

H0: M1 = M2

There is no difference between the purchasing averages of the control group and the test group.

H1: M1 != M2

There is a difference between the purchasing averages of the control group and the test group.


In [4]:
df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
Control,550.89406
Test,582.10610


We do not know whether this difference statistically actually exists. To demonstrate this the reality of the difference, we must perform hypothesis testing.

**Performing Hypothesis Testing**

Before testing the hypothesis, check the assumptions. These are Assumption of Normality and Homogeneity of Variance.

**Normality Assumption**

H0: Normal distribution assumption is provided.

H1: Normal distribution assumption not provided

p < 0.05 H0 REJECTABLE

p > 0.05 H0 CANNOT BE REJECTED

In [6]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "Control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


H0 CANNOT BE REJECTED

In [7]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "Test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


H0 CANNOT BE REJECTED

We can say that the values of the control and test groups provide the assumption of normal distribution.

**Variance Homogeneity**

H0: Variances are homogeneous.

H1: Variances are not homogeneous.

p < 0.05 H0 REJECTABLE

p > 0.05 H0 CANNOT BE REJECTED

Both assumptions are provided so, we'll use Parametric T-test

**Independent two-sample t-test (Parametric test)**

In [10]:
test_stat, pvalue = ttest_ind(df.loc[df["group"] == "Control", 'Purchase'],
                           df.loc[df["group"] == "Test", 'Purchase'],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


p-value > 0.05 

H0 CANNOT BE REJECTED

There is no real purchase difference between, maximum and average bidding types.